In [34]:
import pandas as pd
import re
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [10]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=['target','id', 'date','query','user','tweet'])

In [19]:
df.head()

,target,id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [20]:
df['target'].value_counts()

4    800000
0    800000
Name: target, dtype: int64

In [16]:
X = df['tweet']
y = df['target']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 45)

In [21]:
X_train.shape

(1120000,)

In [22]:
X_test.shape

(480000,)

# Tweets Preprocessing

In [25]:
# Remove hashtags, urls, emoticons, punctuations

In [29]:
import preprocessor as p

In [30]:
def processtweets(data):
    return p.clean(data)
    

In [31]:
pro = X_train.apply(processtweets)

In [33]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [54]:
pro[2]

'I dived many times for the ball. Managed to save % The rest go out of bounds'

In [66]:
X_train.head(10)

257464     a shit jimmie johnson won  not tht i hate him ...
190051        @Lyndzei I don't know...  insomnia probably...
345500     @kimberlybrooke Oh no! I hope you get everythi...
489921     Well today went absolutly not the way it was s...
1413980    Nthaaaaw, miley and nick is writing a song tog...
783686     Grrr @skabbster woke me up at 3:45 when he got...
446890                    margret is leaving the apprentice 
1593495    Good Morning. Just sipping my morning coffee g...
1433378                 enjoying some quality time w my bro 
263934     Woah woah woah.  Susan Boyle's apparently been...
Name: tweet, dtype: object

In [67]:
# REFERENCE : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [123]:
def preprocess2(tweet):
    # Remove URLS and mentions @
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet) 
    
    # Make all characters lower case
    tweet = tweet.lower()
    
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    
    # Code a way to separate hashtags if together
    tweet = [char for char in tweet if char not in string.punctuation]
    tweet = ''.join(tweet)
    tweet = remove_emoji(tweet)
    
    tweet = [ps.stem(word) for word in tweet] # Stem each word
    tweet = ''.join(tweet) 
    
    return tweet
    

In [124]:
preprocess2("@switchfoot http://twitpic.com/2y1zl - Awww, th🙌at's a bummer.  #bro#bro#broham You shoulda got David Carr of Third Day to do it. ;D👋 ")

'  awww thats a bummer brobrobroham you shoulda got david carr of third day to do it d '

In [125]:
t ="this is a test adsfs@fdsf"
t = ''.join(re.sub('@', "", t))
print(t)

this is a test adsfsfdsf


In [126]:
remove_emoji("@switchfoot http://twitpic.com/2y1zl - Awww, th🙌 at's a bummer.  You shoulda got David Carr of Third Day to do it. ;D👋 ")

"@switchfoot http://twitpic.com/2y1zl - Awww, th at's a bummer.  You shoulda got David Carr of Third Day to do it. ;D "

In [127]:
X_train.head()

257464     a shit jimmie johnson won  not tht i hate him ...
190051        @Lyndzei I don't know...  insomnia probably...
345500     @kimberlybrooke Oh no! I hope you get everythi...
489921     Well today went absolutly not the way it was s...
1413980    Nthaaaaw, miley and nick is writing a song tog...
Name: tweet, dtype: object

In [128]:
clean_text = X_train.apply(preprocess2)

In [129]:
clean_text.shape

(1120000,)

In [130]:
print(clean_text[4])

 no its not behaving at all im mad why am i here because i cant see you all over there 


In [114]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [136]:
clean_text_test = X_test.apply(preprocess2)
print(clean_text_test.shape)


(480000,)


In [168]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

In [157]:
tfidfvect = TfidfVectorizer()

In [158]:
vectors = tfidfvect.fit_transform(clean_text)

In [159]:
nb = MultinomialNB()


In [160]:
nb.fit(vectors, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [161]:
test_vectors = tfidfvect.transform(clean_text_test)

In [163]:
test_vectors.shape

(480000, 346504)

In [170]:
print('Score: \t',nb.score(vectors,y_train))
y_pred = nb.predict(vectors)
print('Confustion Matrix \n' ,confusion_matrix(y_train, y_pred))

Score: 	 0.8176419642857143
Confustion Matrix 
 [[470969  89242]
 [114999 444790]]


In [169]:
print('Score: \t',nb.score(test_vectors,y_test))
y_pred = nb.predict(test_vectors)
print('Confustion Matrix \n' ,confusion_matrix(y_test, y_pred))

Score: 	 0.7727958333333333
Confustion Matrix 
 [[191504  48285]
 [ 60773 179438]]


In [171]:
import pickle

model_filename = "naive_bayes_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(nb, file)
    
vectorizer_filename = "count_vect.pkl"
with open(vectorizer_filename, 'wb') as file:
    pickle.dump(tfidfvect, file)

In [174]:
d = 'congratulations you have won 1 million'
d = d.lower()
d = [d]
d = [ps.stem(word) for word in d if word]
nb.predict(['this is awesome'])

ValueError: Expected 2D array, got 1D array instead:
array=['this is awesome'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.